In [1]:
###############################################
# The most common way to initialize a model   #
###############################################

import gurobipy as gp
from gurobipy import GRB

# Create a new model
m = gp.Model (" mip1 ")
# Create variables
# For multiple vars use addVars
# For multiple constraints use addConstrs
x = m.addVar ( vtype =GRB.BINARY , name ="x")
y = m.addVar ( vtype =GRB.BINARY , name ="y")
z = m.addVar ( vtype =GRB.BINARY , name ="z")
# Add constraint : x + 2 y + 3 z <= 4
m.addConstr (x + 2 * y + 3 * z <= 4, "c0")
# Add constraint : x + y >= 1
m.addConstr (x + y >= 1, "c1")

# Set objective
m.setObjective (x + y + 2 * z, GRB.MAXIMIZE )
# Optimize model

Using license file c:\gurobi911\gurobi.lic


In [2]:
m.optimize()

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 2 rows, 3 columns and 5 nonzeros
Model fingerprint: 0xf43f5bdf
Variable types: 0 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective 2.0000000
Presolve removed 2 rows and 3 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 12 available processors)

Solution count 2: 3 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.000000000000e+00, best bound 3.000000000000e+00, gap 0.0000%


In [3]:
#See the solution of a certain var
for var in m.getVars():
    # name of the var
    if 'x' == str(var.VarName[0]):
        print(var.x)

1.0


In [5]:
###########################################
# initialize a model using sparse matrix  #
###########################################

import gurobipy as gp
from gurobipy import GRB
import numpy as np
import scipy.sparse as sp

m=gp.Model("matrix1")
x = m.addMVar ( shape =3, vtype =GRB.BINARY , name ="x")
obj = np. array ([1.0 , 1.0 , 2.0])
m.setObjective (obj @ x, GRB. MAXIMIZE )
# Build ( sparse ) constraint matrix
# (row,col) --> val
val = np. array ([1.0 , 2.0 , 3.0 , -1.0 , -1.0])
row = np. array ([0 ,    0,    0,    1,     1])
col = np. array ([0 ,    1,    2,    0,     1])
A = sp.csr_matrix (( val , (row , col )), shape =(2 , 3)) #row and col means the index of the values
rhs = np. array ([4.0 , -1.0])
# Add constraints
m.addConstr (A @ x <= rhs , name ="c")
# Optimize model
m.optimize ()
print (x.X)



Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 2 rows, 3 columns and 5 nonzeros
Model fingerprint: 0x6d8be88c
Variable types: 0 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective 2.0000000
Presolve removed 2 rows and 3 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds
Thread count was 1 (of 12 available processors)

Solution count 2: 3 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.000000000000e+00, best bound 3.000000000000e+00, gap 0.0000%
[1. 0. 1.]


In [6]:
m.getVars()

[<gurobi.Var x[0] (value 1.0)>,
 <gurobi.Var x[1] (value 0.0)>,
 <gurobi.Var x[2] (value 1.0)>]

In [7]:
#######################################################
#  An Example of the use of tuplelist and select function #
#######################################################

from gurobipy import tuplelist
Cities= [('A','B'), ('A','C'), ('B','C'),('B','D'),('C','D')]
Routes = tuplelist(Cities)
print(Routes.select('A','*'))

<gurobi.tuplelist (2 tuples, 2 values each):
 ( A , B )
 ( A , C )
>


In [8]:
###########################################################################
#   This example reads a model from a file, sets up a callback that       #
#   monitors optimization progress and implements a custom                #
#   termination strategy, and outputs progress information to the         #
#   screen and to a log file.                                             #
#                                                                         # 
#   The termination strategy implemented in this callback stops the       # 
#   optimization of a MIP model once at least one of the following two    #
#   conditions have been satisfied:                                       #
#     1) The optimality gap is less than 10%                              #
#     2) At least 10000 nodes have been explored, and an integer feasible #
#        solution has been found.                                         #
#   Note that termination is normally handled through Gurobi parameters   #
#   (MIPGap, NodeLimit, etc.).  You should only use a callback for        #
#   termination if the available parameters don't capture your desired    # 
#   termination criterion.                                                #
###########################################################################
import sys
import gurobipy as gp
from gurobipy import GRB


# Define my callback function

def mycallback(model, where):
    if where == GRB.Callback.POLLING:
        # Ignore polling callback
        pass
    elif where == GRB.Callback.PRESOLVE:
        # Presolve callback
        cdels = model.cbGet(GRB.Callback.PRE_COLDEL)
        rdels = model.cbGet(GRB.Callback.PRE_ROWDEL)
        if cdels or rdels:
            print('%d columns and %d rows are removed' % (cdels, rdels))
    elif where == GRB.Callback.SIMPLEX:
        # Simplex callback
        itcnt = model.cbGet(GRB.Callback.SPX_ITRCNT)
        if itcnt - model._lastiter >= 100:
            model._lastiter = itcnt
            obj = model.cbGet(GRB.Callback.SPX_OBJVAL)
            ispert = model.cbGet(GRB.Callback.SPX_ISPERT)
            pinf = model.cbGet(GRB.Callback.SPX_PRIMINF)
            dinf = model.cbGet(GRB.Callback.SPX_DUALINF)
            if ispert == 0:
                ch = ' '
            elif ispert == 1:
                ch = 'S'
            else:
                ch = 'P'
            print('%d %g%s %g %g' % (int(itcnt), obj, ch, pinf, dinf))
    elif where == GRB.Callback.MIP:
        # General MIP callback
        nodecnt = model.cbGet(GRB.Callback.MIP_NODCNT)
        objbst = model.cbGet(GRB.Callback.MIP_OBJBST)
        objbnd = model.cbGet(GRB.Callback.MIP_OBJBND)
        solcnt = model.cbGet(GRB.Callback.MIP_SOLCNT)
        if nodecnt - model._lastnode >= 100:
            model._lastnode = nodecnt
            actnodes = model.cbGet(GRB.Callback.MIP_NODLFT)
            itcnt = model.cbGet(GRB.Callback.MIP_ITRCNT)
            cutcnt = model.cbGet(GRB.Callback.MIP_CUTCNT)
            print('%d %d %d %g %g %d %d' % (nodecnt, actnodes,
                  itcnt, objbst, objbnd, solcnt, cutcnt))
        if abs(objbst - objbnd) < 0.1 * (1.0 + abs(objbst)):
            print('Stop early - 10% gap achieved')
            model.terminate()
        if nodecnt >= 10000 and solcnt:
            print('Stop early - 10000 nodes explored')
            model.terminate()
    elif where == GRB.Callback.MIPSOL:
        # MIP solution callback
        nodecnt = model.cbGet(GRB.Callback.MIPSOL_NODCNT)
        obj = model.cbGet(GRB.Callback.MIPSOL_OBJ)
        solcnt = model.cbGet(GRB.Callback.MIPSOL_SOLCNT)
        x = model.cbGetSolution(model._vars)
        print('**** New solution at node %d, obj %g, sol %d, '
              'x[0] = %g ****' % (nodecnt, obj, solcnt, x[0]))
    elif where == GRB.Callback.MIPNODE:
        # MIP node callback
        print('**** New node ****')
        if model.cbGet(GRB.Callback.MIPNODE_STATUS) == GRB.OPTIMAL:
            x = model.cbGetNodeRel(model._vars)
            model.cbSetSolution(model.getVars(), x)
    elif where == GRB.Callback.BARRIER:
        # Barrier callback
        itcnt = model.cbGet(GRB.Callback.BARRIER_ITRCNT)
        primobj = model.cbGet(GRB.Callback.BARRIER_PRIMOBJ)
        dualobj = model.cbGet(GRB.Callback.BARRIER_DUALOBJ)
        priminf = model.cbGet(GRB.Callback.BARRIER_PRIMINF)
        dualinf = model.cbGet(GRB.Callback.BARRIER_DUALINF)
        cmpl = model.cbGet(GRB.Callback.BARRIER_COMPL)
        print('%d %g %g %g %g %g' % (itcnt, primobj, dualobj,
              priminf, dualinf, cmpl))
    elif where == GRB.Callback.MESSAGE:
        # Message callback
        msg = model.cbGet(GRB.Callback.MSG_STRING)
        model._logfile.write(msg)


